In [2]:
# Setting up the notebook
import sys
import os

# Import local libraries
sys.path.append('../exercises_complements')
from shared_functions import check_task_with_solution

# Exercise 3 : Darcy experiment


## Context : 

A Darcy experiement set-up is installed.

You are given the following data :
    - Hydraulic conductivity K = 0.0003 $\text{m}/\text{s}^{-1}$
    - Column diameter d = 2.54 cm
    - Effective porosity $\phi_e$ = 0.25
    - Head difference is  $\Delta h$ = 0.4 m
    - Column length L = 1 m




## Tasks

In [3]:

# Launch the tasks
check_task_with_solution("task03_1")
check_task_with_solution("task03_2")
check_task_with_solution("task03_3")



## Task 1:
Based on given data, estimate the specific discharge $q$ : 


FloatText(value=0.0, description='Answer:', style=DescriptionStyle(description_width='initial'))

Button(description='Submit', style=ButtonStyle())

Output()

Button(description='Show Solution', disabled=True, style=ButtonStyle())

...

FloatText(value=0.0, description='Answer:', style=DescriptionStyle(description_width='initial'))

Button(description='Submit', style=ButtonStyle())

Output()

Button(description='Show Solution', disabled=True, style=ButtonStyle())

Output()